Propósito: Compara el precio del ticker con el precio objetivo, después el "fair value" segun el análisis fundamental(formula de Graham).
Puede fallar si se analiza un ticker con alta volatilidad. Arroja una alerta si el analisis arroja un precio inflado o "burbuja" que podría explotar en cualquier momento.

In [ ]:
#Extrae los datos del ticker solicitado.
import yfinance as yf
TICKER = "PLTR"
stock = yf.Ticker(TICKER)

In [ ]:
# Seccion de Value Investing (Fórmula de Graham). Devuelve la comparativa entre los tres precios.

# --- Precio de cierre ---
info = stock.info
precio_actual = info.get('currentPrice', info.get('previousClose'))

# --- Precio de analistas (Target Price) ---
precio_analistas = info.get('targetMeanPrice')

# ---Fair Value(Fórmula de Graham simplificada)---
info = stock.info
precio_actual = info.get('currentPrice')
precio_analistas = info.get('targetMeanPrice')

# Extraemos el EPS (Beneficio por acción)
eps = info.get('trailingEps')

# Extrae el precio de "g" growth rate (tasa de crecimiento a largo plazo).
g = info.get('longTermAverageGrowthRate', 0.15) # Valor por defecto 0,15 si no se encuentra el dato.
if g < 1: g = g * 100   #  Filtro de datos, si el dato viene como decimal lo convertimos a porcentaje.

# Usamos la fórmula: EPS * (8.5 + 2g) dónde g es la tasa de crecimiento a largo plazo. el EPS el beneficio por acción.
fair_value_graham = round(eps * (8.5 + 2 * g), 2)

# Guardamos estos valores del fundamental.
analisis_fundamental = {
    'Actual': precio_actual,
    'Analistas': precio_analistas,
    'FairValue': fair_value_graham
    }

print(f"ANALISIS SOBRE {TICKER}:")
print(f"1) Precio Mercado:  {precio_actual}")
print(f"2) Precio Objetivo: {precio_analistas}")
print(f"3) Valor Graham:    {fair_value_graham}")

In [ ]:
#Detector de precios burbuja.

p_graham = analisis_fundamental['FairValue'] 
p_mercado = analisis_fundamental['Actual']   

#Cálculo del ratio entre el Valor de Graham y el Precio de Mercado. Si el Valor de Graham es menos del 60% del precio de mercado,
# significa que el precio está inflado un 40% o más por encima de sus fundamentos.
def detectar_burbuja(p_mercado, p_graham):
    
    ratio_valor = p_graham / p_mercado # Compara qué tanto del precio está respaldado por valor real
    
    #definimos el umbral de "burbuja": Un ratio < 0.60 significa que el 40% del precio es pura especulación (aire).
    if ratio_valor < 0.60: 
        print(f"¡ALERTA DE BURBUJA!: El valor real de la empresa {TICKER} de USD(${p_graham}) solo cubre el {round(ratio_valor*100)}% del precio.") # Notifica la brecha
        return True
    else:
        print(f"VALUACIÓN SANA: El fundamento respalda el {round(ratio_valor*100)}% del precio.")
        return False


alerta_activa = detectar_burbuja(p_mercado, p_graham) # Ejecuta la función

# --- AUDITORÍA TÉCNICA FINAL --
eps_audit = analisis_fundamental.get('EPS', 1) # Evitamos división por cero con 1
per_calc = p_mercado / eps_audit
g_req = (per_calc - 8.5) / 2

if g_req > 30:
    print(f"Para justificar el precio de ${p_mercado}, la empresa debería crecer al {round(g_req)}% anual.")


In [ ]:
#Gráfico de barras comparativo: precio actual vs precio objetivo vs precio Graham
import matplotlib.pyplot as plt

# 1. Preparar los datos del diccionario
nombres = list(analisis_fundamental.keys())
valores = list(analisis_fundamental.values())

# 2. Crear el gráfico
plt.figure(figsize=(10, 6))
barras = plt.bar(nombres, valores, color=['slategray', 'royalblue', 'crimson'])

# 3. Estética simple
plt.title(f'Comparativa de Valuación: {TICKER}', fontsize=14, fontweight='bold')
plt.ylabel('Precio en USD')
plt.grid(axis='y', linestyle='--', alpha=0.7)


# Añadir los números encima de las barras para que se lean fácil
for barra in barras:
    yval = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2, yval, f'${yval}', 
             va='bottom', ha='center', fontweight='bold')

plt.show()

In [ ]:
#Gráfico de Línea de Tiempo comparativo: precio de mercado (volatilidad) vs precio objetivo vs precio Graham
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use('dark_background') # Cambia el fondo a negro y los textos a blanco

# Obtenemos solo el histórico de precios.
h_precios = stock.history(period="max")

# Recuperamos las variables del diccionario de la celda anterior
p_actual = analisis_fundamental['Actual']
p_analistas = analisis_fundamental['Analistas']
p_graham = analisis_fundamental['FairValue']

# Gráfico de línea de tiempo.
plt.figure(figsize=(12, 6))

#Línea de volatilidad (Precios de cierre)
plt.plot(h_precios.index, h_precios['Close'], label='Precio de Mercado', color='white', alpha=0.8)

#Líneas punteadas denotan el fair value(Graham) y el precio objetivo de analistas.
plt.axhline(y=p_graham, color='crimson', linestyle='--', label=f'Fair Value Graham (${p_graham})')
plt.axhline(y=p_analistas, color='royalblue', linestyle='--', label=f'Analistas (${p_analistas})')

plt.title(f'Análisis de Valor Histórico: {TICKER}', fontsize=14, fontweight='bold')
plt.ylabel('Precio USD')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.2)

#Lógica visual de alerta por burbuja.
if alerta_activa:
    mensaje_visual = "⚠️ALERTA: PRECIO BURBUJA" 
    color_alerta = 'red' 
else:
    mensaje_visual = "VALUACIÓN SANA DEL TICKER"
    color_alerta = 'green' 

# Texto de alerta por burbuja.
plt.text(0.5, 0.95, mensaje_visual, 
         color=color_alerta,           
         fontsize=14,                   
         fontweight='bold',             
         ha='center',                  
         va='center',                    
         transform=plt.gca().transAxes,  
         bbox=dict(facecolor='white', alpha=0.8, edgecolor=color_alerta))

plt.show()